<a href="https://colab.research.google.com/github/CHOKKAKULA1999/Deep-Learning-/blob/main/hackton_deepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORTANT LIBARRIES 

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Data Visualization
import matplotlib.pyplot as plt
import plotly.express as px


ACTUALL CODE START 

In [4]:
seg_train ='C:\\Users\\chokkakula.n\\Downloads\\archive\\seg_train\\seg_train'
seg_test = 'C:\\Users\\chokkakula.n\\Downloads\\archive\\seg_test\\seg_test'
seg_pred = 'C:\\Users\\chokkakula.n\\Downloads\\archive\\seg_pred\\seg_pred'

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        seg_train,  # This is the path to the training directory
        target_size=(150, 150),  # Resize all images to 150x150
        batch_size=32,
        class_mode='categorical')  # Use categorical_crossentropy loss

validation_generator = test_datagen.flow_from_directory(
        seg_test,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        seg_pred,
        target_size=(150, 150),
        batch_size=32,
        class_mode='categorical')



FileNotFoundError: ignored

EACH CLASS DISTRIBUTION 

In [ ]:
class_names = sorted(os.listdir(seg_train))
n_classes = len(class_names)

# Show
print(f"Num Classes : {n_classes}")
print(f"Class Names : {class_names}")

CHECKING THE IMAGES 

In [ ]:
label_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
import matplotlib.pyplot as plt

# Get a batch of images and labels from the train_generator
images, labels = next(train_generator)

# Plot the first 10 images in the batch
fig, ax = plt.subplots(nrows=2, ncols=5, figsize=(10, 6))
for i in range(10):
    row = i // 5
    col = i % 5
    ax[row, col].imshow(images[i])
    ax[row, col].set_title(label_names[np.argmax(labels[i])])
    ax[row, col].axis('off')
plt.tight_layout()
plt.show()

DEFINING ANF BUILDING THE MODEL 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape of the images
input_shape = (150, 150, 3)

# Create a Sequential model
model = Sequential()

# Add the first convolutional layer with 32 filters, a 3x3 kernel size, and ReLU activation
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))

# Add a max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a second convolutional layer with 64 filters and a 3x3 kernel size
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add a max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Add a third convolutional layer with 128 filters and a 3x3 kernel size
model.add(Conv2D(128, (3, 3), activation='relu'))

# Add a max pooling layer with a 2x2 pool size
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Add a fully connected layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Add a dropout layer with a rate of 0.5
model.add(Dropout(0.5))

# Add the output layer with 6 units and softmax activation (for multi-class classification)
model.add(Dense(6, activation='softmax'))

# Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Print the summary of the model architecture
model.summary()


In [ ]:

model.fit(train_generator,
          epochs=10,
          validation_data=validation_generator)

In [ ]:
from PIL import Image
import numpy as np

# Load an image using PIL
img = Image.open('C:\\Users\\chokkakula.n\\Downloads\\archive\\seg_pred\\seg_pred\\388.jpg')

# Resize the image to a target size
img = img.resize((150, 150))

# Convert the PIL image to a NumPy array
img_array = np.array(img)

# Preprocess the image array
img_array = img_array.astype('float32') / 255.0
img_array = img_array.reshape((1, 150, 150, 3))

# Make a prediction using the model
prediction = model.predict(img_array)
# Get the predicted class label
predicted_class = np.argmax(prediction)

print('Predicted class:', predicted_class)


In [ ]:
import csv
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array



# Define the path to the directory containing the images
image_dir = 'C:\\Users\\chokkakula.n\\Downloads\\archive\\seg_pred\\seg_pred'

# Get the list of image filenames in the directory
image_filenames = os.listdir(image_dir)

# Create a list to hold the predictions
predictions = []

# Loop over the images and make predictions
for filename in image_filenames:
    # Load the image
    img = load_img(os.path.join(image_dir, filename), target_size=(150, 150))
    
    # Convert the image to a NumPy array
    img_array = img_to_array(img)
    
    # Normalize the pixel values
    img_array /= 255.
    
    # Reshape the array to match the input shape of the model
    img_array = img_array.reshape((1, ) + img_array.shape)
    
    # Make the prediction
    pred = model.predict(img_array)
    
    # Get the predicted class label
    predicted_class = np.argmax(pred, axis=1)[0]
    
    # Add the prediction to the list
    predictions.append((filename, predicted_class))

# Write the predictions to a CSV file
with open('predictions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['image name', 'predicted class'])
    for prediction in predictions:
        writer.writerow(prediction)

In [ ]:
# Save the model to a file
model.save('my_model.h5')

BY USING VGG16

In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

# Load the VGG16 model without the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the weights of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)